판례에 등장하는 단어들의 상관관계 분석은 단어들 간의 관계를 분석하여 어떤 단어들이 함께 자주 등장하는지, 그리고 이들이 서로에게 어떤 영향을 미치는지를 파악하는 데 유용합니다

1. 단어-문서 행렬 기반 분석은 단어의 빈도를 기반으로 단순한 상관관계를 파악합니다.
2. TF-IDF 분석은 단어의 중요도에 따라 빈도수 상관관계를 보정하여 더 중요한 단어들 간의 관계를 분석합니다.
3. Word2Vec은 문맥을 기반으로 단어들의 유사도를 분석하여 의미론적 관계까지 분석할 수 있습니다.

1. 단어-문서 행렬 기반 상관관계 분석 (CountVectorizer)

설명: 이 방법은 각 단어가 문서 내에서 얼마나 자주 등장하는지 계산한 후, 그 단어들 간의 상관관계를 측정합니다. 이는 단순히 공분산에 기반하여, 특정 단어들이 같이 등장하는 경향을 파악할 수 있습니다

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 판례 데이터를 불러옴 (예시 데이터)
documents = [
    "형법 제250조에 의한 살인 사건",
    "형법 제250조와 민법 제100조의 충돌",
    "민법 제100조에 따른 상속 분쟁",
    "대법원의 판결에 따른 민법 해석",
    "형법 제250조와 판례 해석"
]

# CountVectorizer로 단어-문서 행렬 생성
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
word_freq_matrix = X.toarray()

# 단어 목록 생성
words = vectorizer.get_feature_names_out()

# 상관관계 계산 (단순한 공분산으로 계산 가능)
correlation_matrix = pd.DataFrame(word_freq_matrix, columns=words).corr()

# 상관관계 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Word Correlation Matrix")
plt.show()


2. TF-IDF 기반 상관관계 분석
TF-IDF(Term Frequency-Inverse Document Frequency)는 단어 빈도를 반영하되, 문서에서 자주 등장하는 흔한 단어의 가중치를 줄여주는 방식으로 상관관계를 분석합니다. 중요한 단어들 간의 관계를 더 잘 파악할 수 있습니다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# TF-IDF Vectorizer로 단어-문서 행렬 생성
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents).toarray()

# 단어 목록 생성
words = tfidf_vectorizer.get_feature_names_out()

# 상관관계 계산
correlation_matrix = pd.DataFrame(tfidf_matrix, columns=words).corr()

# 상관관계 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="Blues", fmt=".2f")
plt.title("TF-IDF Word Correlation Matrix")
plt.show()


설명: 이 방법은 TF-IDF 값을 기반으로 상관관계를 분석합니다. 이는 빈도가 높아도 중요하지 않은 단어들(예: '그리고', '하지만')을 자동으로 배제하면서 중요한 단어들 간의 상관관계를 더 잘 드러내 줍니다.

3. 협업 필터링 기반 상관관계 분석 (Word2Vec)
Word2Vec과 같은 협업 필터링 알고리즘을 이용해 단어들 간의 유사도를 계산하고 상관관계를 분석할 수 있습니다. Word2Vec은 단어의 문맥을 바탕으로 벡터를 생성하고, 이를 이용해 단어 간의 유사성을 측정합니다.

In [ ]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# 판례 문서를 형태소로 나누는 전처리 (간단히 공백 기준)
tokenized_docs = [doc.split() for doc in documents]

# Word2Vec 모델 학습
model = Word2Vec(sentences=tokenized_docs, vector_size=100, window=5, min_count=1, workers=4)

# 상관관계 분석할 단어 목록 선택
words_of_interest = ["형법", "민법", "제250조", "제100조", "살인", "상속"]

# 단어 벡터들 추출
word_vectors = [model.wv[word] for word in words_of_interest]

# 단어 간의 유사도(코사인 유사도) 계산
similarities = model.wv.most_similar("형법", topn=10)  # 예시로 '형법'과 가장 유사한 단어들 추출

# 유사한 단어들을 출력
print(similarities)

# 유사도에 따라 단어 벡터 간 상관관계 계산 및 시각화
pca = PCA(n_components=2)
result = pca.fit_transform(word_vectors)

plt.figure(figsize=(10, 8))
sns.scatterplot(x=result[:, 0], y=result[:, 1], s=100)

for i, word in enumerate(words_of_interest):
    plt.text(result[i, 0], result[i, 1], word, fontsize=12)

plt.title("Word2Vec Word Correlation")
plt.show()


설명: Word2Vec 모델은 문서 내 단어들의 문맥적 유사성을 기반으로 벡터를 학습합니다. 이를 통해 특정 단어들과 유사한 단어들을 찾고, 그들의 상관관계를 분석할 수 있습니다. 이 방법은 단어 간의 의미론적 관계를 파악하는 데 유용합니다.